In [39]:
from urllib.request import urlopen
import re
from datetime import datetime

In [51]:
current_month = datetime.now().month
current_year = datetime.now().year

months = [((current_month + i-1) % 12) + 1 for i in range(3)]

years = [current_year for i in range(3)]

if current_month >= 11:
    years[2] = current_year + 1
    if current_month == 12:
        years[1] = current_year + 1

months, years

([1, 2, 3], [2024, 2024, 2024])

In [56]:
def make_url(month, year):
    month_str = str(month)
    month_str = month_str.rjust(2, '0')
    year_str = str(year)
    url = 'https://www.revivalhubla.com/film-calendar?view=calendar&month=' + month_str + '-' + year_str
    return url

urls = [make_url(month, year) for month, year in zip(months, years)]
pages = [urlopen(url) for url in urls]
htmls = [page.read().decode("utf-8") for page in pages]

In [57]:
pattern = "<h1>.*?</h1>"
results = [re.findall(pattern, html) for html in htmls]

In [36]:
def extract_text(line):
    html_chunks = re.findall("<.*?>", line)
    for chunk in html_chunks:
        line = line.replace(chunk, '')
    return line

In [59]:
titles = [list(map(extract_text, result)) for result in results]

titles

[['The Favourite - 2018',
  'Fish Tank - 2009',
  'The Little Foxes - 1941 / Jezebel - 1938',
  'Open Mic Short Night - January 2024',
  'Golden Years (Die goldenen Jahre) - 2022',
  'The Wizard of Oz - 1939 (85th Anniversary)',
  'The Breakfast Club - 1985',
  '"Bad Timing" - 1982 / Household Saints - 1993 (New 4K Restoration!)',
  'Luminous Procuress - 1971 (+ short film The Yellow Sequence) (pres. by Lightstruck)',
  'Christine - 1983',
  'Cry-Baby - 1990',
  'The Little Foxes - 1941 / Jezebel - 1938',
  'B.A.P.S. - 1997 / Booty Call - 1997',
  'The 5 Minutes Game presented by The Museum of Home Video',
  'Inside the Yellow Cocoon Shell - 2023',
  'The Wizard of Oz - 1939 (85th Anniversary)',
  'To Die For - 1995',
  'Fragments of Paradise - 2022 {Q&amp;A w/ filmmaker K.D. Davison}',
  'All About Lily Chou-Chou - 2001',
  'Navigators - 2022 (LA Premiere pres. by Mezzanine) {Q&amp;A w/ filmmaker Noah Teichner}',
  'Brokeback Mountain - 2005',
  'Mami Wata - 2023 {Q&amp;A with filmmak